In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import os
import datetime
import pickle
from tqdm import tqdm
import lxml
from pykml.factory import KML_ElementMaker as KML

In [2]:
data_dir = './data/'
file_prefixes = ['NO_TS_MO', 'BO_TS_MO']

write_folder = './all_data_KML'
kml_pinmap_fn = 'kml_pinmap'

file_filter = [
    'BO_TS_MO_Dalbosjon.nc',
    'BO_TS_MO_Koster.nc',
    'BO_TS_MO_Vaderobod.nc',
    'BO_TS_MO_Vaderoarna1.nc',
    'BO_TS_MO_Borno.nc',
    'BO_TS_MO_Havstensfjord.nc',
    'BO_TS_MO_HavstensfjordBS.nc',
    'BO_TS_MO_Kristineberg1.nc',
    'BO_TS_MO_Kristineberg2.nc',
    'BO_TS_MO_Tangesund.nc',
    'NO_TS_MO_BuoyHoernumTief1.nc',
    'NO_TS_MO_Hoernum.nc',
    'NO_TS_MO_HelgolandWR.nc',
    'NO_TS_MO_CuxhavenFerryBox.nc',
    'NO_TS_MO_PileSpiekeroog.nc',
    'NO_TS_MO_MeetboeiRZGN1.nc',
    'NO_TS_MO_Borkum.nc',
    'NO_TS_MO_MeetboeiUHW1.nc',
    'NO_TS_MO_Uithuizerwad2.nc',
    'NO_TS_MO_Uithuizerwad3.nc',
    'NO_TS_MO_SchiermonnikoogWaggen.nc',
    'NO_TS_MO_Wierumerwad3.nc',
    'NO_TS_MO_Wierumerwad2.nc',
    'NO_TS_MO_Amelander31.nc',
    'NO_TS_MO_Amelander32.nc',
    'NO_TS_MO_Amelander41.nc',
    'NO_TS_MO_Amelander42.nc',
    'NO_TS_MO_Amelander51.nc',
    'NO_TS_MO_Amelander52.nc',
    'NO_TS_MO_Amelander61.nc',
    'NO_TS_MO_Amelander62.nc',
    'NO_TS_MO_StortemelkOost.nc',
    'NO_TS_MO_StortemelkBoei.nc',
    'NO_TS_MO_Pannengat1.nc',
    'NO_TS_MO_Breesem1.nc',
    'NO_TS_MO_KornwerderzandBuiten.nc',
    'NO_TS_MO_Breezanddijkbuiten1.nc',
    'NO_TS_MO_Malzwin1.nc',
    'NO_TS_MO_MeetboeiPBW1.nc',
    'NO_TS_MO_DenHelder.nc',
    'NO_TS_MO_Ijmuiden.nc',
    'NO_TS_MO_IJmondstroompaal.nc',
    'NO_TS_MO_IJgeulstroompaal1.nc',
    'NO_TS_MO_IJmondstroompaal2.nc',
    'NO_TS_MO_MaeslantkeringZeezijdeNoordMeetpaal.nc',
    'NO_TS_MO_MaeslantkeringZeezijdeZuidMeetpaal.nc',
    'NO_TS_MO_HoekVanHolland.nc',
    'NO_TS_MO_HoekVanHollandNAP.nc',
    'NO_TS_MO_zdv8-2d-MMND.nc',
    'NO_TS_MO_Brouwershavensegat8.nc',
    'NO_TS_MO_Oosterschelde4.nc',
    'NO_TS_MO_Roompotsluis.nc',
    'NO_TS_MO_KeetenBoei.nc',
    'NO_TS_MO_MarollegatMeetpaal.nc',
    'NO_TS_MO_Hansweert.nc',
    'NO_TS_MO_HansweertPunt2.nc',
    'NO_TS_MO_Bath.nc',
    'NO_TS_MO_OverloopVanValkenisse.nc',
    'NO_TS_MO_PasVanTerneuzenBoei.nc',
    'NO_TS_MO_Hoofdplaat.nc',
    'NO_TS_MO_HoofdplaatMeetpaal2.nc',
    'NO_TS_MO_HonteSloehaven.nc',
    'NO_TS_MO_ZwinBuoy.nc',
    'NO_TS_MO_ZeebruggeDam.nc',
    'NO_TS_MO_ZeebruggePortBuoy.nc',
    'NO_TS_MO_ZeebruggeWeatherStation.nc',
    'NO_TS_MO_Blankenberge.nc',
    'NO_TS_MO_OostendEasternPalisade.nc',
    'NO_TS_MO_OostendBuoy.nc',
    'NO_TS_MO_OostendWeather.nc',
    'NO_TS_MO_Raversijde.nc',
    'NO_TS_MO_NieuwpoortWind.nc',
    'NO_TS_MO_Roscoff.nc',
    'NO_TS_MO_Neustadt.nc',
    'NO_TS_MO_Kuehlungsborn.nc',
    'NO_TS_MO_BuoyHoernumTief1.nc',
    'NO_TS_MO_BuoyBunkerHill1.nc',
    'NO_TS_MO_HirtshalsWR.nc',
    'NO_TS_MO_HanstholmWR.nc',
    'NO_TS_MO_BuoyBunkerHill1.nc',
    'NO_TS_MO_BuoyHoernumTief1.nc',
    'NO_TS_MO_CadzandBoei.nc',
    'NO_TS_MO_BolVanHeist.nc',
    'NO_TS_MO_A2.nc',
    'NO_TS_MO_NieuwpoortBuoy.nc',
    'NO_TS_MO_6200303.nc',
    'BO_TS_MO_Varberg.nc',
    'BO_TS_MO_Svinbadan,nc',
    'BO_TS_MO_GreatBeltBridgeEastCU.nc',
]

files = [f for f in os.listdir(data_dir) if any([f.startswith(fp) for fp in file_prefixes if f not in file_filter])]

df = pd.DataFrame({'file':pd.Series(dtype=object), 'LONG':pd.Series(dtype=float), 'LAT':pd.Series(dtype=float)}).rename_axis('file')

for file in tqdm(files):
    #TODO
    #result_df = search_file_fixed_params(file, result_df)
    ds = xr.open_dataset(os.path.join(data_dir, file))
    
    #Take only average location since some bouys are non-stationary
    ave_long = np.average(ds.coords['LONGITUDE'].values)
    ave_lat = np.average(ds.coords['LATITUDE'].values)
    df.loc[len(df.index)] = [file,ave_long,ave_lat]
    

#Conditionally creates the folder for the result
os.makedirs(write_folder, exist_ok=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 166/166 [00:06<00:00, 26.04it/s]


In [3]:
df

,file,LONG,LAT
file,,,
0,BO_TS_MO_Svinbadan.nc,12.516698,56.166691
1,NO_TS_MO_IJgeul5.nc,4.268056,52.493046
2,NO_TS_MO_OostendNorth.nc,2.883000,51.351994
3,NO_TS_MO_6300113.nc,1.708000,61.000000
4,NO_TS_MO_DarsserSCU.nc,12.699998,54.700008
...,...,...,...
161,NO_TS_MO_FINO1WR.nc,6.583334,54.000000
162,NO_TS_MO_WielingenNoord.nc,3.415498,51.433372
163,BO_TS_MO_Flinten7BS.nc,12.844300,55.591419


In [6]:
i, row = next(df.iterrows())
row['LAT']

KeyError: 'LAT'

In [5]:
#Create separate kml map
KML_fldr = KML.Folder(
    KML.name("All files"),
    #KML.description(),
)

#iterate over all unique variable name, file name pairs
for index,row in df.iterrows():
    #extract longitude and latitude
    long = row['LONG']
    lat = row['LAT']
    
    #Create the pin
    pin = KML.Placemark(
        KML.name(row['file']),
        KML.description(row['file']),
        KML.Point(
            KML.coordinates(str(long) + "," + str(lat))
        )
    )
    KML_fldr.append(pin)
    
with open(os.path.join(write_folder, kml_pinmap_fn + '.kml'), 'w') as f_w:
    f_w.write(lxml.etree.tostring(KML_fldr, pretty_print=True).decode())